In [ ]:
import os
import javabridge
import bioformats
import numpy as np
from tifffile import imwrite

def lif_to_tiff(input_path, output_dir, ome_tiff=False):
    # Start the Java Virtual Machine
    javabridge.start_vm(class_path=bioformats.JARS)

    # Read the .lif file with bioformats
    r = bioformats.ImageReader(input_path)
    
    # Determine number of series (individual images) in the .lif file
    n_series = r.rdr.getSeriesCount()

    for series in range(n_series):
        r.rdr.setSeries(series)
        
        # Get image dimensions
        x, y, z, c, t = (r.rdr.getSizeX(), r.rdr.getSizeY(), r.rdr.getSizeZ(), 
                         r.rdr.getSizeC(), r.rdr.getSizeT())

        # Initialize an empty array for image data
        img_data = np.empty((t, z, y, x, c), dtype=np.uint16)

        # Fill the array with image data
        for ti in range(t):
            for zi in range(z):
                for ci in range(c):
                    img_data[ti, zi, :, :, ci] = r.read(z=zi, t=ti, c=ci, rescale=False)

        # Save as TIFF or OME-TIFF
        filename = f"image_series_{series}"
        filepath = os.path.join(output_dir, filename + ('.ome.tiff' if ome_tiff else '.tiff'))
        imwrite(filepath, img_data)

    # Close the Java Virtual Machine
    javabridge.kill_vm()


input_lif = "/Users/katherineridley/Dropbox (UK Dementia Research Institute)/KRidley/PlaqueStack/Lifs/F_2_M04_OC.lif"
output_directory = "/Users/katherineridley/Dropbox (UK Dementia Research Institute)/KRidley/PlaqueStack/OMETIFFs"

lif_to_tiff(input_lif, output_directory, ome_tiff=True)
'''if __name__ == "__main__":
    input_folder = "/Users/katherineridley/Dropbox (UK Dementia Research Institute)/KRidley/PlaqueStack/Lifs"
    output_directory = "/Users/katherineridley/Dropbox (UK Dementia Research Institute)/KRidley/PlaqueStack/OMETIFFs"



    for input_lif in os.listdir(input_folder):
        if input_lif.endswith('lif'):
            lif_to_tiff(input_lif, output_directory, ome_tiff=True)'''


In [ ]:
import os
import subprocess

series=0
# Function to get the number of series in a .lif file
def get_series_count(lif_file, bfconvert_path):
    command = f"{bfconvert_path} -noflat {lif_file}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    output = result.stdout

    # Debugging: Print the output to see what's being captured
    print("bfconvert output:")
    print(output)

    series_count = sum(1 for line in output.split('\n') if line.startswith("Series "))
    return series_count


# Paths
input_directory = "/Users/katherineridley/APPFIRE_image/Lifs"
output_directory = "/Users/katherineridley/APPFIRE_image/OMETIFFs"
bfconvert_path = "/Users/katherineridley/Downloads/bftools/bfconvert" 

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate through all .lif files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".lif"):
        input_file = os.path.join(input_directory, filename)
        series_count = get_series_count(input_file, bfconvert_path)
        print (filename, series_count)

        # Loop through each series and channel
        
        for channel in [0, 1]:
                output_file = os.path.join(output_directory, "%n_c{}.ome.tiff".format(channel))
                command = f"{bfconvert_path} -noflat -channel {channel} {input_file} {output_file}"
                subprocess.run(command, shell=True)

                series =+1





In [ ]:
import cv2
import numpy as np
import tifffile as tiff
from skimage import io

def apply_clahe(image, clip_limit=1.0, tile_grid_size=(8, 8)):
    # Convert the image to a format suitable for CLAHE
    image = image.astype(np.uint8)
    # Create a CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    # Apply CLAHE to the image
    return clahe.apply(image)

def export_max_projections_with_clahe(ome_tiff_path, output_dir):
    with tiff.TiffFile(ome_tiff_path) as tif:
        max_projection_ch1 = None
        max_projection_ch2 = None

        for series in tif.series:
            stack = series.asarray()
            ch1 = stack[:, 0, :, :]
            ch2 = stack[:, 1, :, :]

            mip_ch1 = np.max(ch1, axis=0)
            mip_ch2 = np.max(ch2, axis=0)

            if max_projection_ch1 is None:
                max_projection_ch1 = mip_ch1
                max_projection_ch2 = mip_ch2
            else:
                max_projection_ch1 = np.maximum(max_projection_ch1, mip_ch1)
                max_projection_ch2 = np.maximum(max_projection_ch2, mip_ch2)

        # Normalize and apply CLAHE
        max_projection_ch1 = apply_clahe(((max_projection_ch1 / max_projection_ch1.max()) * 255).astype(np.uint8))
        max_projection_ch2 = apply_clahe(((max_projection_ch2 / max_projection_ch2.max()) * 255).astype(np.uint8))

        # Save the max intensity projections of each channel
        io.imsave(f"{output_dir}/max_projection_ch1_clahe.tiff", max_projection_ch1)
        io.imsave(f"{output_dir}/max_projection_ch2_clahe.tiff", max_projection_ch2)

        # Create an RGB image for the overlay
        rgb_image = np.zeros((max_projection_ch1.shape[0], max_projection_ch1.shape[1], 3), dtype=np.uint8)
        rgb_image[..., 0] = max_projection_ch1  # Red
        rgb_image[..., 1] = max_projection_ch2  # Green
        rgb_image[..., 2] = max_projection_ch1  # Blue

        # Save the RGB max intensity projection with CLAHE
        io.imsave(f"{output_dir}/max_projection_rgb_clahe.tiff", rgb_image)

# Example usage
export_max_projections_with_clahe('/Users/katherineridley/testtiff2_B_2_M04_OC_DGG_P+_031123_2.ome.tiff', 
    '/Users/katherineridley/Dropbox (UK Dementia Research Institute)/KRidley/imaging/example images/'
)


In [ ]:
conda install pyimagej


In [ ]:
import imagej
import os
import imagej
import scyjava as sj

# initialize ImageJ


# import ImgPlus class


#/Library/Java/JavaVirtualMachines/jdk-<version>.jdk/Contents/Home.

# Set JAVA_HOME to the path of your Java installation
java_path = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'  # Replace with your Java path
os.environ['JAVA_HOME'] = java_path

# Initialize ImageJ
ij = imagej.init('/Users/katherineridley/Downloads/Fiji.app')

Ops = sj.jimport('net.imagej.ops.Ops')

def max_intensity_projection(ome_tiff_path, output_path):
    # Use ImageJ to open the OME-TIFF file
    imp = ij.io().open(ome_tiff_path)

    # Check if the image was successfully opened
    if imp is None:
        print("Could not open image from path:", ome_tiff_path)
        return

    
    #n_channels = 2

    #for ch in range(n_channels):
        # Create a ZProjector for max intensity projection
        # Note: ij.op().run(...) is used to run ImageJ operations
    max_op = ij.op().op(Ops.Stats.Max, imp)

    # Run the project operation
    # Here, `imp` is the input, 2 is the Z-dimension, `projection` is the output
    # `max_op` is the operation to perform
    projection = ij.op().run(Ops.Transform.Project, imp, 2, max_op)
    

    


    # Prepare output path for the channel
    ch_output_path = os.path.splitext(output_path)[0] + '_ch{}.tiff'.format(1)
    
    # Save the projection using ImageJ's utility
    ij.io().save(projection, ch_output_path)
    print("Saved channel", ch + 1, "projection to", ch_output_path)



# Example usage
ome_tiff_path = '/Users/katherineridley/testtiff2_B_2_M04_OC_DGG_P+_031123_2.ome.tiff'
output_path = '/Users/katherineridley/testimagejz.tiff'
max_intensity_projection(ome_tiff_path, output_path)


In [ ]:

projected_dimension = 'Z'
# Select which dimension to project
data = ij.io().open(ome_tiff_path)

print(data)

Ops = sj.jimport('net.imagej.ops.Ops')
Axes = sj.jimport('net.imagej.axis.Axes')

dim = data.dimensionIndex(getattr(Axes, 'Z'))
 
if dim == -1:
    raise Exception("%s dimension not found." % projected_dimension)
 
if data.dimension(dim) < 2:
    raise Exception("%s dimension has only one frame." % projected_dimension)
 
# Write the output dimensions
new_dimensions = [data.dimension(d) for d in range(0, data.numDimensions()) if d != dim]
 
# Create the output image
projected = Ops.create().img(new_dimensions)
 
# Create the op and run it
proj_op = Ops.op(getattr(Ops.Stats, 'Max'), data)
Ops.transform().project(projected, data, proj_op, dim)
 
# Create the output Dataset
output = ds.create(projected)

In [ ]:
pip install --use-pep517 pyjnius

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

image_path = '/Users/katherineridley/testtiff2_B_2_M04_OC_DGG_P+_031123_2.ome.tiff'
IM = io.imread(path)
IM_MAX= np.max(IM, axis=0)

print(IM_MAX)
#plt.imshow(IM_MAX)

In [ ]:
pip install scikit-image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io
import tifffile
image_path = '/Users/katherineridley/testtiff2_B_2_M04_OC_DGG_P+_031123_2.ome.tiff'
stack = skimage.io.imread(image_path)
MAXdata = np.max(stack, axis=0)


tifffile.imwrite("/Users/katherineridley/testStack1_max.ome.tiff", MAXdata, compression='zlib') 
#tifffile.imwrite("/Users/katherineridley/testStack1_avg.tif", AVGdata, compression='zlib') 
#tifffile.imwrite("/Users/katherineridley/testStack1_avg16.tif", AVGdata16, compression='zlib') 

In [ ]:

import pywt
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_float

def wavelet_denoise(image, mode='soft', wavelet='db1', level=1):
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    threshold = np.sqrt(2*np.log(image.size)) * np.median(np.abs(coeffs[-level])) / 0.6745
    new_coeffs = map(lambda x: pywt.threshold(x, value=threshold, mode=mode), coeffs)
    return pywt.waverec2(list(new_coeffs), wavelet)

# Load image
image_path = "/Users/katherineridley/testStack1_max.ome.tiff"
image = img_as_float(io.imread(image_path))

# If the image is in color, convert it to grayscale
if len(image.shape) == 3:
    image = rgb2gray(image)

# Apply wavelet denoising
denoised_image = wavelet_denoise(image)

# Display the original and denoised images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(denoised_image, cmap='gray')
axes[1].set_title('Denoised Image')
axes[1].axis('off')

plt.show()

# Save the denoised image
output_path = "/Users/katherineridley/"
io.imsave(output_path, denoised_image)

In [ ]:
import cv2

def apply_clahe(image, clip_limit=1.0, tile_grid_size=(8, 8)):
    # Convert the image to a format suitable for CLAHE
    image = image.astype(np.uint8)
    # Create a CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    # Apply CLAHE to the image
    return clahe.apply(image)

image_path = "/Users/katherineridley/testStack1_max.ome.tiff"
image = img_as_float(io.imread(image_path))

apply_clahe(((image/ image.max()) * 255).astype(np.uint8))

In [ ]:
pip install opencv-python

In [ ]:
import os
import numpy as np
import skimage.io
import tifffile

def process_image(image_path, output_path):
    stack = skimage.io.imread(image_path)
    MAXdata = np.max(stack, axis=0)
    tifffile.imwrite(output_path, MAXdata, compression='zlib')

def merge_channels(c0_path, c1_path, output_path):
    c0_image = skimage.io.imread(c0_path)
    c1_image = skimage.io.imread(c1_path)
    merged_image = np.maximum(c0_image, c1_image)  # Simple overlay by taking max
    tifffile.imwrite(output_path, merged_image, compression='zlib')

input_directory = '/Users/katherineridley/APPFIRE_image/OMETIFFs'
output_directory = '/Users/katherineridley/APPFIRE_image/MaxProjections'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

processed_files = {}  # To keep track of processed files for merging

# Loop through all .ome.tiff files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".ome.tiff"):
        # Splitting filename to get base name and channel
        parts = filename.split('_')
        print(parts)
        channel = parts[-1].split('.')[0] 
        base_name = '_'.join(parts[:-1])

        image_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, f"{base_name}_{channel}_maxproj.ome.tiff")

        process_image(image_path, output_path)

        # Store the output paths for merging
        if base_name not in processed_files:
            processed_files[base_name] = {}
        processed_files[base_name][channel] = output_path

# Merge channels for each base image
for base_name, channels in processed_files.items():
    if 'c0' in channels and 'c1' in channels:
        merge_channels(channels['c0'], channels['c1'], 
                       os.path.join(output_directory, f"{base_name}_merged_maxproj.ome.tiff"))


In [15]:
import os
import numpy as np
import skimage.io
import tifffile

def process_image(image_path, output_path):
    print('image path within function', image_path)
    stack = skimage.io.imread(image_path, is_ome=False)
    #print(stack)
    MAXdata = np.max(stack, axis=0)

    print('writing 1')
    tifffile.imwrite(output_path, MAXdata)

def merge_channels(c0_path, c1_path, output_path):
    c0_image = skimage.io.imread(c0_path)
    c1_image = skimage.io.imread(c1_path)

    # Initialize merged image with zeros
    merged_image = np.zeros((*c0_image.shape, 3), dtype=c0_image.dtype)

    # Assign channels (c0 in magenta, c1 in green)
    merged_image[..., 0] = c0_image  # Red channel for c0
    merged_image[..., 2] = c0_image  # Blue channel for c0
    merged_image[..., 1] = c1_image  # Green channel for c1

    print('writing 2')
    tifffile.imwrite(output_path, merged_image)

input_directory = '/Users/katherineridley/APPFIRE_image/OMETIFFs'
output_directory = '/Users/katherineridley/APPFIRE_image/MaxProjections'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

processed_files = {}  # To keep track of processed files for merging

# Loop through all .ome.tiff files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".ome.tiff"):
        # Splitting filename to get base name and channel
        parts = filename.split('_')
        channel = parts[-1] 
        c = channel.split('.')[0]
        #print(c, 'here is channel') # Assuming channel is the second last element
        base_name = '_'.join(parts[:-1])
        print('base', base_name)
        print('filename here', input_directory, filename)
        image_path = os.path.join(input_directory, filename)
        output_path = os.path.join(output_directory, f"{base_name}_{c}_maxproj.ome.tiff")

        process_image(image_path, output_path)

        # Store the output paths for merging
        if base_name not in processed_files:
            processed_files[base_name] = {}
            processed_files[base_name][c] = output_path
            print('output path is', output_path)
            print('image path is', image_path)
print('output path is', output_path)
print('image path is', image_path)
# Merge channels for each base image
for base_name, channels in processed_files.items():
    print('both together at merge', base_name, channels)
    if 'c0' in channels and 'c1' in channels:
        print('merging occurs')
        merge_channels(channels['c0'], channels['c1'], 
                       os.path.join(output_directory, f"{base_name}_merged_maxproj.ome.tiff"))


base C_2_M04_OC_CA1_P+_161123_1
filename here /Users/katherineridley/APPFIRE_image/OMETIFFs C_2_M04_OC_CA1_P+_161123_1_c0.ome.tiff
image path within function /Users/katherineridley/APPFIRE_image/OMETIFFs/C_2_M04_OC_CA1_P+_161123_1_c0.ome.tiff
writing 1
output path is /Users/katherineridley/APPFIRE_image/MaxProjections/C_2_M04_OC_CA1_P+_161123_1_c0_maxproj.ome.tiff
image path is /Users/katherineridley/APPFIRE_image/OMETIFFs/C_2_M04_OC_CA1_P+_161123_1_c0.ome.tiff
base C_2_M04_OC_CA1_P+_161123_1
filename here /Users/katherineridley/APPFIRE_image/OMETIFFs C_2_M04_OC_CA1_P+_161123_1_c1.ome.tiff
image path within function /Users/katherineridley/APPFIRE_image/OMETIFFs/C_2_M04_OC_CA1_P+_161123_1_c1.ome.tiff
writing 1
base B_3_M04_OC_DGG_P+_131123_1
filename here /Users/katherineridley/APPFIRE_image/OMETIFFs B_3_M04_OC_DGG_P+_131123_1_c1.ome.tiff
image path within function /Users/katherineridley/APPFIRE_image/OMETIFFs/B_3_M04_OC_DGG_P+_131123_1_c1.ome.tiff
writing 1
output path is /Users/kather